<a href="https://colab.research.google.com/github/Zhaoxin-Hu/ECE222A/blob/master/Homework_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt

import numpy as np
np.set_printoptions(suppress=True)

from mpl_toolkits import mplot3d
import mpl_toolkits.mplot3d.axes3d as axes3d

from scipy import special
from scipy import integrate
from scipy.signal import argrelextrema

## Problem 2

Antenna Size ($2^n \times 2^n$)

In [0]:
# directivity
n = np.array([1, 2, 3, 4, 5])
Ap = (2**n)**2
D = 4*np.pi*Ap
D_dB = 10*np.log10(D)
print('D (dB): ', D_dB)

# efficiency (loss)
freq = 3e9
c = 3e8
n = 1
wvlen = c/freq*1e2 # in cm
loss_per_cm = 0.1
loss_factor = 0
for i in range(n):
    loss_factor = loss_factor + 2*2**(i-1)*wvlen*loss_per_cm
print(loss_factor)

D (dB):  [17.01269855 23.03329847 29.05389838 35.07449829 41.09509821]
1.0


## Problem 3

In [0]:
er = 1
b = 0.5/np.sqrt(er)
a = 1.5*b
dx = a
dy = b
k = 2*np.pi

'''num_pts_theta = 200
num_pts_phi = 200
theta, phi = np.linspace(0, np.pi/2, num_pts_theta), np.linspace(0, 2*np.pi, num_pts_phi)
theta2d, phi2d = np.meshgrid(theta, phi)
theta2d, phi2d = np.transpose(theta2d), np.transpose(phi2d)'''

In [0]:
def AF_un(theta, phi):
    psiy = k*dy*np.sin(theta)*np.sin(phi)
    AFy = 1 + np.exp(1j*psiy)
    return np.abs(AFy)

def EF(theta, phi):
    psix = k*dx*np.sin(theta)*np.cos(phi)
    return np.abs(np.sqrt(np.sin(phi)**2+(np.cos(theta)*np.cos(phi))**2)*np.sinc(psix/2))

def P_un(theta, phi):
    return (AF_un(theta, phi)*EF(theta,phi))**2
  
P_un_max = P_un(theta2d, phi2d).max()

def P_n(theta, phi):
    return P_un(theta, phi)/P_un_max

def integrand(theta, phi):
    return P_n(theta, phi)*np.sin(theta)

In [0]:
num_pts_theta = 500
theta = np.linspace(-np.pi, np.pi, num_pts_theta)
plt.figure(figsize = (5,5))
ax = plt.subplot(111, projection='polar')
ax.set_theta_offset(np.pi/2)
ax.set_theta_direction(-1)
ax.plot(theta, 10*np.log10(P_n(theta, np.pi/2)))
ax.set_rmax(0)
ax.set_rmin(-30)
ax.set_title(r'YZ-plane Pattern (dB angular plot)', y=1.1)
ax.set_xlabel(r'$\theta$')